## Ch09 대규모 데이터의 처리를 위한 Sparse matrix 사용

이 책에서 약 1600개의 영화에 대한 100000개의 평점을 포함하고 있는 데이터이다.  
현실에서 다루어야 하는 데이터는 이보다 훨씬 더 크다.  
Movielens 20M 데이터는 약 2000만 개의 데이터를 포함하고 있는데 이런 데이터를 처리할 때 흔하게 발생하는 문제는 메모리의 한계이다.  
앞에서는 데이터를 full matrix로 변환하여 처리하였는데 이 데이터를 같은 방식을 적용하면 너무 많은 원소의 행렬이 생기게 되어 메모리의 한계로 처리할 수 없다.  
이 때 사용하는 것이 sparse matrix 로 이는 추천 시스템에서 사용되는 대부분의 데이터에서 full matrix로 변환했을 때 많은 원소가 비어있는 희박행렬을 말한다.

### 9.1 Sparse matrix의 개념과 Python에서의 사용

다룰 데이터가 sparse matrix 라면 전체 행렬을 저장하는 대신 행렬의 원소 중에서 실제로 값을 갖는 것만 인덱스와 같이 저장하는 것이 더 효율적이다.  
이 방식은 데이터가 희박할수록 저장공간이 절약되는 정도가 커진다.

sparse matrix의 단점은 데이터를 저장하거나 읽어올 때마다 값이 존재하는지 확인해서 그에 맞는 처리를 해야 하기 때문에 데이터 처리의 오버헤드 비용이 크다.  
따라서 데이터가 희박하지 않은 경우 sparse matrix를 사용하면 속도저하의 단점이 크게 부각되므로 데이터가 희박한 경우에만 사용하는 것이 낫다.

다음은 sparse matrix 간단한 사용 예이다.

In [4]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

ratings = {'user_id' : [1,2,4],
           'movie_id' : [2,3,7],
           'rating' : [4,3,1]}
ratings = pd.DataFrame(ratings)

In [5]:
# Pandas pivot 을 이용해 full matrix로 변환하는 경우
rating_matrix = ratings.pivot(index = 'user_id', columns = 'movie_id',
                              values = 'rating').fillna(0)

full_matrix1 = np.array(rating_matrix)
print(full_matrix1)

[[4. 0. 0.]
 [0. 3. 0.]
 [0. 0. 1.]]


In [9]:
# Sparse matrix를 이용해서 full matrix로 변환하는 경우
data = np.array(ratings['rating'])
row_idx = np.array(ratings['user_id'])
col_idx = np.array(ratings['movie_id'])
rating_matrix = csr_matrix((data, (row_idx, col_idx)), dtype=int)
print(rating_matrix)

full_matrix2 = rating_matrix.toarray()
print(full_matrix2)

  (1, 2)	4
  (2, 3)	3
  (4, 7)	1
[[0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0]
 [0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]]


In [11]:
# sparse matrix 계산
print(rating_matrix * 2, '\n')
print(rating_matrix.T, '\n')
print(rating_matrix.dot(rating_matrix.T))

  (1, 2)	8
  (2, 3)	6
  (4, 7)	2 

  (2, 1)	4
  (3, 2)	3
  (7, 4)	1 

  (1, 1)	16
  (2, 2)	9
  (4, 4)	1


### 9.2 Sparse matrix 추천 알고리즘에 적용하기

대규모 데이터에 기존의 방식을 적용하면 앞서 말한 거대한 행렬이 되기 때문에 Python에서 해결할 수 없어 다음과 같은 에러가 발생한다.  
__Unstacked DataFrame is too big, causing int32 overflow__

이를 sparse matrix를 적용해 해결할 수 있다.

In [13]:
# 데이터 읽기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', names = r_cols, sep = '\t')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

In [14]:
# train test 분리
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=1)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

In [15]:
# Sparse matrix 사용
from scipy.sparse import csr_matrix
data = np.array(ratings['rating'])
row_idx = np.array(ratings['user_id'])
col_idx = np.array(ratings['movie_id'])
ratings = csr_matrix((data, (row_idx, col_idx)), dtype=int)

In [ ]:
# New MF class for training & testing
class NEW_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = ratings
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # train set의 RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Test set을 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):              # test 데이터에 있는 각 데이터에 대해서
            x, y, z = ratings_test.iloc[i]              # user_id, item_id, rating 가져오기
            test_set.append([x, y, z])
            self.R[x, y] = 0                            # Setting test set ratings to 0
        self.test_set = test_set
        return test_set                                 # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))

    # Training 하면서 test set의 정확도를 계산
    def test(self):
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(user_id, item_id)

# Testing MF RMSE
R_temp = ratings.copy()
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=190, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

코드를 실행하는데 매우 긴 시간이 걸리지만 Sparse matrix를 사용하면 numpy array로는 불가능한 매우 큰 규모의 데이터를 분석할 수 있다는 장점이 있다.